Ich habe im Notebook "Gewinnsteuer zusammenfügen" das Problem gehabt, dass ab 2010 mein vorheriger Code nicht mehr funktioniert, weil die Kolonne drei zwei Zahlen beinhaltet und durch /n getrennt ist. Dieses Problem versuche ich hier zu lösen.

In [86]:
import camelot
import tabula
import pandas as pd

In [87]:
Firmen_2010 = camelot.read_pdf("Firmen_2010.pdf", flavor='stream', pages='all', table_areas=['0,842,595,0'])

In [88]:
#zurechtschneiden
test_df = Firmen_2010[0].df
Firmen_10 = test_df[12:]
firmen_2010 = Firmen_10.head(5)
firmen_2010

,0,1,2,3,4,5
12,,0,10'000,21'101\n \n1'051'519,6'782,116'360
13,,10'100,50'000,4'270\n \n8'379'718,279,427'696
14,,50'100,250'000,3'302\n \n35'558'173,77,448'731
15,,250'100,+,2'028\n \n388'354'588,34,1'503'531
16,,Total,,30'701\n \n433'343'998,7'172,2'496'318


In [89]:

#headers setzen
headers = ['Weglassen', 'Steuerbarer Gewinn ab CHF', 'weglassen', 'etwas seltsames',
           'lassweg', 'überflüssig']
firmen_2010.columns = headers
firmen_2010

,Weglassen,Steuerbarer Gewinn ab CHF,weglassen,etwas seltsames,lassweg,überflüssig
12,,0,10'000,21'101\n \n1'051'519,6'782,116'360
13,,10'100,50'000,4'270\n \n8'379'718,279,427'696
14,,50'100,250'000,3'302\n \n35'558'173,77,448'731
15,,250'100,+,2'028\n \n388'354'588,34,1'503'531
16,,Total,,30'701\n \n433'343'998,7'172,2'496'318


In [90]:
# firmen_2010['A'], firmen_2010['B'] = firmen_2010['etwas seltsames'].str.split('\').str
# von https://stackoverflow.com/questions/14745022/how-to-split-a-column-into-two-columns
# gibt errormeldung

### So erhalte ich (nach 2 Stunden ausprobieren) aus der einen Kolonne zwei Kolonnen

In [91]:
getrennte_kolonnen = firmen_2010['etwas seltsames'].str.split(' ', 1, expand=True) 
# von https://stackoverflow.com/questions/14745022/how-to-split-a-column-into-two-columns
getrennte_kolonnen

,0,1
12,21'101\n,\n1'051'519
13,4'270\n,\n8'379'718
14,3'302\n,\n35'558'173
15,2'028\n,\n388'354'588
16,30'701\n,\n433'343'998


In [92]:
#firmen_2010['Anzahl Steuerpflichtige 2010'], firmen_2010['Betrag Kantonssteuer 2010 in CHF'] = firmen_2010['etwas seltsames'].str.split(' ', 1, expand=True) 
#fügt mir nur zwei kolonnen hinten an

In [93]:
firmen_2010

,Weglassen,Steuerbarer Gewinn ab CHF,weglassen,etwas seltsames,lassweg,überflüssig
12,,0,10'000,21'101\n \n1'051'519,6'782,116'360
13,,10'100,50'000,4'270\n \n8'379'718,279,427'696
14,,50'100,250'000,3'302\n \n35'558'173,77,448'731
15,,250'100,+,2'028\n \n388'354'588,34,1'503'531
16,,Total,,30'701\n \n433'343'998,7'172,2'496'318


### Ich versuche, firmen_2010 mit getrennte_kolonnen zu mergen

In [94]:
del firmen_2010['Weglassen']
del firmen_2010['weglassen']
del firmen_2010['lassweg']
del firmen_2010['etwas seltsames']

In [95]:
firmen_2010

,Steuerbarer Gewinn ab CHF,überflüssig
12,0,116'360
13,10'100,427'696
14,50'100,448'731
15,250'100,1'503'531
16,Total,2'496'318


In [96]:
del firmen_2010['überflüssig']

In [97]:
firmen_2010

,Steuerbarer Gewinn ab CHF
12,0
13,10'100
14,50'100
15,250'100
16,Total


In [98]:
#pd.merge(dfunfälle,einwohnerzahlenneu,
        # how ='left',on='Gemeinde')

In [99]:
pd.merge(firmen_2010, getrennte_kolonnen, 
         how = right, left_index = False)  #, left_index=False, right_index=False

NameError: name 'right' is not defined

## das funktioniert nicht, weil ich keine gemeinsame kolonne habe der beiden. ich überlege mir ernsthaft, eine neue kolonne hinzuzufügen. so wie vorhin. 

In [100]:
firmen_2010['alibi kolonne'] = firmen_2010['Steuerbarer Gewinn ab CHF'].str.split(' ', 1, expand=True) 

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [101]:
firmen_2010

,Steuerbarer Gewinn ab CHF,alibi kolonne
12,0,0
13,10'100,10'100
14,50'100,50'100
15,250'100,250'100
16,Total,Total


In [102]:
getrennte_kolonnen['alibi kolonne'] = getrennte_kolonnen['Steuerbarer Gewinn ab CHF'].str.split(' ', 1, expand=True) 

KeyError: 'Steuerbarer Gewinn ab CHF'

### funktioniert nicht. 

In [103]:
firmen_2010['Steuerbarer Gewinn ab CHF'] = pd.Series(getrennte_kolonnen.random.randn(sLength), index=firmen_2010.index)

AttributeError: 'DataFrame' object has no attribute 'random'

In [104]:
getrennte_kolonnen['Steuerbarer Gewinn ab CHF'] = firmen_2010.values

ValueError: Wrong number of items passed 2, placement implies 1

## Step back. Nochmals von vorne mit diesen beiden Dataframes hier

In [105]:
firmen_2010

,Steuerbarer Gewinn ab CHF,alibi kolonne
12,0,0
13,10'100,10'100
14,50'100,50'100
15,250'100,250'100
16,Total,Total


In [106]:
getrennte_kolonnen

,0,1
12,21'101\n,\n1'051'519
13,4'270\n,\n8'379'718
14,3'302\n,\n35'558'173
15,2'028\n,\n388'354'588
16,30'701\n,\n433'343'998


In [107]:
#pd.merge(firmen_2010, getrennte_kolonnen,
        #how ='left', on = "Index")
    
#das hatte nicht funktioniert

## Versuch mit Concat
defunden hier dank dem Hinweis auf zip in einem älteren Notebook https://stackoverflow.com/questions/34318141/zip-pandas-dataframes-into-a-new-dataframe

In [108]:
Firmen_2010_neu = pd.concat([firmen_2010, getrennte_kolonnen], axis=1)
Firmen_2010_neu

,Steuerbarer Gewinn ab CHF,alibi kolonne,0,1
12,0,0,21'101\n,\n1'051'519
13,10'100,10'100,4'270\n,\n8'379'718
14,50'100,50'100,3'302\n,\n35'558'173
15,250'100,250'100,2'028\n,\n388'354'588
16,Total,Total,30'701\n,\n433'343'998


In [109]:
#headers setzen
headers = ['Steuerbarer Gewinn ab CHF', 'streichen', 'Anzahl Steuerpflichtige 2010', 'Betrag Kantonssteuer 2010 in CHF']
Firmen_2010_neu.columns = headers
Firmen_2010_neu

,Steuerbarer Gewinn ab CHF,streichen,Anzahl Steuerpflichtige 2010,Betrag Kantonssteuer 2010 in CHF
12,0,0,21'101\n,\n1'051'519
13,10'100,10'100,4'270\n,\n8'379'718
14,50'100,50'100,3'302\n,\n35'558'173
15,250'100,250'100,2'028\n,\n388'354'588
16,Total,Total,30'701\n,\n433'343'998


In [110]:
del Firmen_2010_neu['streichen']
Firmen_2010_neu

,Steuerbarer Gewinn ab CHF,Anzahl Steuerpflichtige 2010,Betrag Kantonssteuer 2010 in CHF
12,0,21'101\n,\n1'051'519
13,10'100,4'270\n,\n8'379'718
14,50'100,3'302\n,\n35'558'173
15,250'100,2'028\n,\n388'354'588
16,Total,30'701\n,\n433'343'998


Jetzt muss ich putzen, und zwar etwas mehr als vorhin

In [111]:
def geputzt(zeile):
    zeile = zeile.replace('(', '')
    zeile = zeile.replace(')', '')
    zeile = zeile.replace('\n', '') #das neu rein
    zeile = zeile.replace("'", "").strip()
    return zeile
Firmen_2010_neu['Steuerbarer Gewinn ab CHF'] = Firmen_2010_neu['Steuerbarer Gewinn ab CHF'].apply(geputzt)
Firmen_2010_neu['Anzahl Steuerpflichtige 2010'] = Firmen_2010_neu['Anzahl Steuerpflichtige 2010'].apply(geputzt)
Firmen_2010_neu['Betrag Kantonssteuer 2010 in CHF'] = Firmen_2010_neu['Betrag Kantonssteuer 2010 in CHF'].apply(geputzt)

In [112]:
Firmen_2010_neu

,Steuerbarer Gewinn ab CHF,Anzahl Steuerpflichtige 2010,Betrag Kantonssteuer 2010 in CHF
12,0,21101,1051519
13,10100,4270,8379718
14,50100,3302,35558173
15,250100,2028,388354588
16,Total,30701,433343998


# Juhuiii, ich hab die Chose gepackt! Nach fast einem Tag Arbeit!

### morgen schaue ich, wie ich von hier weiter gehe. Achtung: Den Code oben werde ich voraussichtlich auch bei den Einkommenssteuern brauchen können. Dort hab ich dieselbe Chose gesichtet

In [113]:
firmen07 = Firmen_2010_neu.T
firmen07

,12,13,14,15,16
Steuerbarer Gewinn ab CHF,0,10100,50100,250100,Total
Anzahl Steuerpflichtige 2010,21101,4270,3302,2028,30701
Betrag Kantonssteuer 2010 in CHF,1051519,8379718,35558173,388354588,433343998


Jetzt bin ich am selben punkt wie vorhin. Ich fahre also weiter mit meinem Code wie bisher.

In [114]:
firmen_10_neu = Firmen_2010_neu.T
firmen_10_neu

,12,13,14,15,16
Steuerbarer Gewinn ab CHF,0,10100,50100,250100,Total
Anzahl Steuerpflichtige 2010,21101,4270,3302,2028,30701
Betrag Kantonssteuer 2010 in CHF,1051519,8379718,35558173,388354588,433343998


In [116]:
Firmen10 = firmen_10_neu[1:]
Firmen10

,12,13,14,15,16
Anzahl Steuerpflichtige 2010,21101,4270,3302,2028,30701
Betrag Kantonssteuer 2010 in CHF,1051519,8379718,35558173,388354588,433343998


In [117]:
headers = ['Steuerbarer Gewinn 0-10000 CHF', 'Steuerbarer Gewinn 10100-50000 CHF', 'Steuerbarer Gewinn 50100-250000 CHF', 
           'Steuerbarer Gewinn ab 250100 CHF', 'Total Steuerbarer Gewinn']
Firmen10.columns = headers
Firmen10

,Steuerbarer Gewinn 0-10000 CHF,Steuerbarer Gewinn 10100-50000 CHF,Steuerbarer Gewinn 50100-250000 CHF,Steuerbarer Gewinn ab 250100 CHF,Total Steuerbarer Gewinn
Anzahl Steuerpflichtige 2010,21101,4270,3302,2028,30701
Betrag Kantonssteuer 2010 in CHF,1051519,8379718,35558173,388354588,433343998


Und noch die Tabellen auseinander nehmen

In [118]:
Betrag_Kantonssteuer_2010 = Firmen10.drop(['Anzahl Steuerpflichtige 2010'])
Betrag_Kantonssteuer_2010

,Steuerbarer Gewinn 0-10000 CHF,Steuerbarer Gewinn 10100-50000 CHF,Steuerbarer Gewinn 50100-250000 CHF,Steuerbarer Gewinn ab 250100 CHF,Total Steuerbarer Gewinn
Betrag Kantonssteuer 2010 in CHF,1051519,8379718,35558173,388354588,433343998


In [119]:
Anzahl_Steuerpflichtige_2010 = Firmen10.drop(['Betrag Kantonssteuer 2010 in CHF'])
Anzahl_Steuerpflichtige_2010

,Steuerbarer Gewinn 0-10000 CHF,Steuerbarer Gewinn 10100-50000 CHF,Steuerbarer Gewinn 50100-250000 CHF,Steuerbarer Gewinn ab 250100 CHF,Total Steuerbarer Gewinn
Anzahl Steuerpflichtige 2010,21101,4270,3302,2028,30701


So. Jetzt baue ich diesen Code zusammen im Notebook "Gewinnsteuer Zusammenfügen von Jahren"